<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Eye_ai_Image_Cropping_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install git+https://github.com/informatics-isi-edu/deriva-py
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools' --upgrade --force
!pip install 'git+https://github.com/informatics-isi-edu/deriva-ml' --upgrade --force
!pip install git+https://github.com/informatics-isi-edu/eye-ai-ml
!pip install pydantic --upgrade --force

import json
import os
from eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath


In [ ]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


In [ ]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id )

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid="2-5NZC")
configuration_records


In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
bag_path1 = configuration_records['bag_paths'][0]
Dataset_Path = PurePath(bag_path1, 'data/Image.csv')
Dataset = pd.read_csv(Dataset_Path)

Dataset_Field_2 = Dataset[Dataset['Image_Angle_Vocab'] == "2SK6"]

# For sanity check, compare the filtered data length by just taking the Filename column and filtering it with the same condition where it has Field 2 in it as a substring.

print(len(Dataset_Field_2))
print(len(Dataset[Dataset['Filename'].str.contains("Field 2")]))

file2_csv_path = PurePath(bag_path1, 'Field_2.csv')
Dataset_Field_2.to_csv(file2_csv_path, index=False)

In [ ]:
# @title Execute ML algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
cm = EA.start_execution(execution_rid=configuration_records['execution'])
with cm as exec:
  preprocess_and_crop(
      bag_path1+"/data/assets/Image/",
      file2_csv_path,
      './output/output.csv',
      'template.jpg',
      './output/',
      './'+configuration_records['model_paths'][0],
      configuration_records['execution'],
      configuration_records['annotation_tag_rid'],
      EA.configuration.annotation_tag.name
      )

In [ ]:
# @title ML result analysis
import numpy as np
output_csv_path = "./output/output.csv"
cropped_info = pd.read_csv(output_csv_path)[["Image RID", "Worked Image Cropping Function"]]

cropped_info['Cropped'] = np.where(cropped_info['Worked Image Cropping Function'] == 'Raw Cropped to Eye', 'False', 'True')
cropped_info = cropped_info[["Image RID", "Cropped"]]
cropped_info.rename(columns={'Image RID': 'RID'}, inplace=True)

In [ ]:
# @title Upload ML results
# upload cropping bounding box
EA.upload_assets(f'./output/{configuration_records["execution"]}')

# upload cropping metadata
EA.update_image_table(cropped_info)